In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import the standard tools for pythonic data analysis.
import csv
import math
import random
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

cli=pd.read_csv('/content/gdrive/MyDrive/CS3244 Team 05/Datasets/Energy/train/client_train.csv')
inv=pd.read_csv('/content/gdrive/MyDrive/CS3244 Team 05/Datasets/Energy/train/invoice_train.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
cli

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0
...,...,...,...,...,...,...
135488,62,train_Client_99995,11,304,26/07/2004,0.0
135489,63,train_Client_99996,11,311,25/10/2012,0.0
135490,63,train_Client_99997,11,311,22/11/2011,0.0
135491,60,train_Client_99998,11,101,22/12/1993,0.0


In [ ]:
# Raw Dataframe that combines both client and invoice dataframe together based on their client_id
df=(cli.join(inv.set_index('client_id',), on='client_id')) 

In [ ]:
df.to_csv("raw_train_data.csv") # Run only when exporting raw dataframe

In [ ]:
df.head()

,disrict,client_id,client_catg,region,creation_date,target,invoice_date,tarif_type,counter_number,counter_statue,...,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,60,train_Client_0,11,101,31/12/1994,0.0,2014-03-24,11,1335667,0,...,8,1,82,0,0,0,14302,14384,4,ELEC
0,60,train_Client_0,11,101,31/12/1994,0.0,2013-03-29,11,1335667,0,...,6,1,1200,184,0,0,12294,13678,4,ELEC
0,60,train_Client_0,11,101,31/12/1994,0.0,2015-03-23,11,1335667,0,...,8,1,123,0,0,0,14624,14747,4,ELEC
0,60,train_Client_0,11,101,31/12/1994,0.0,2015-07-13,11,1335667,0,...,8,1,102,0,0,0,14747,14849,4,ELEC
0,60,train_Client_0,11,101,31/12/1994,0.0,2016-11-17,11,1335667,0,...,9,1,572,0,0,0,15066,15638,12,ELEC


In [ ]:
df.dtypes

disrict                   int64
client_id                object
client_catg               int64
region                    int64
creation_date            object
target                  float64
invoice_date             object
tarif_type                int64
counter_number            int64
counter_statue           object
counter_code              int64
reading_remarque          int64
counter_coefficient       int64
consommation_level_1      int64
consommation_level_2      int64
consommation_level_3      int64
consommation_level_4      int64
old_index                 int64
new_index                 int64
months_number             int64
counter_type             object
dtype: object

In [ ]:
FINAL_DF = pd.DataFrame() # Create new empty DataFrame

### Cleaning of Dataset

In [ ]:
# Counter Status
# Drop rows that have Counter Status out of range [0,5]
# df['counter_statue'] = pd.to_numeric(df['counter_statue'], downcast='integer', errors='coerce')
# df = df.drop(df[(df.counter_statue < 0) | (df.counter_statue > 5) | (df.counter_statue.isnull())].index)

df['counter_statue'] = df['counter_statue'].map({0:0,1:1,2:2,3:3,4:4,5:5,769:999,'0':0,'5':5,'1':1,'4':4,'A':999,618:999,269375:999,46:999,420:999})
df = df.drop(df[(df.counter_statue == 999)].index) # Setting invalid data to 999 and droping their rows

# Reading Remark
# Drop rows that have reading remark out of range [0,9]

df = df.drop(df[df.reading_remarque > 9].index)




### Data Type Enforcement 

In [ ]:
# Creation Date. Convert String to Datetime
df['creation_date'] = pd.to_datetime(df['creation_date'])

# Invoice Date. Convert String to Datetime
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

# Target. Convert Float to Integer
df['target'] = pd.to_numeric(df['target'], downcast='integer')

Ensure data types for various features are correct before continuing

In [ ]:
df.dtypes

disrict                          int64
client_id                       object
client_catg                      int64
region                           int64
creation_date           datetime64[ns]
target                            int8
invoice_date            datetime64[ns]
tarif_type                       int64
counter_number                   int64
counter_statue                   int64
counter_code                     int64
reading_remarque                 int64
counter_coefficient              int64
consommation_level_1             int64
consommation_level_2             int64
consommation_level_3             int64
consommation_level_4             int64
old_index                        int64
new_index                        int64
months_number                    int64
counter_type                    object
dtype: object

### Creating Main Attributes / Features

In [ ]:
# Client ID
FINAL_DF['client_id'] = df['client_id'].unique()

# District
FINAL_DF['district'] = (df.groupby('client_id')['disrict'].max()).to_list()

# Client Category 
FINAL_DF['client_catg'] = (df.groupby('client_id')['client_catg'].max()).to_list()

# Region
FINAL_DF['region'] = (df.groupby('client_id')['region'].max()).to_list()

# Region Group
df['region_group'] = df['region'].apply(lambda x: 1 if x<100 else 3 if x>300 else 2)
FINAL_DF['region_group'] = (df.groupby('client_id')['region_group'].max()).to_list()

# Creation Date Day
FINAL_DF['creation_date_day'] = (df.groupby('client_id')['creation_date'].max().dt.day).to_list()

# Creation Date Month
FINAL_DF['creation_date_month'] = (df.groupby('client_id')['creation_date'].max().dt.month).to_list()

# Creation Date Year
FINAL_DF['creation_date_year'] = (df.groupby('client_id')['creation_date'].max().dt.year).to_list()

# Number of Months since creation date
# Max Invoice Date - Client Creation Date
FINAL_DF['no_months_as_client'] = ((df.groupby(["client_id"])['invoice_date'].max() - df.groupby(["client_id"])['creation_date'].min())/np.timedelta64(1, 'M')).to_list()

# Count of Services Consumed
FINAL_DF['services_consumed'] = (df.groupby('client_id')['tarif_type'].nunique()).to_list()

# Fraud/NFraud
FINAL_DF['target'] = (df.groupby('client_id')['target'].max()).to_list()


#########################################

# Index Difference:
df['difference_index'] = df['new_index'] - df['old_index']




### Creating Invoice Data Master Record

In [ ]:
# Months of Service
# Max Invoice Date - Min Invoice Date
FINAL_DF['months_of_service'] = ((df.groupby(["client_id"])['invoice_date'].max() - df.groupby(["client_id"])['invoice_date'].min())/np.timedelta64(1, 'M')).to_list()

# Number of Unique counter 
FINAL_DF['number_of_counter'] = (df.groupby(["client_id"])['counter_number'].nunique()).to_list()

# Number of Invoice Instances
FINAL_DF['number_of_instances'] = (df.groupby(["client_id"])['invoice_date'].count()).to_list()

# Number of Unique person counting
FINAL_DF['number_of_person_counting'] = (df.groupby(["client_id"])['counter_code'].nunique()).to_list()

# Min Reading Remark
FINAL_DF['min_reading_remark'] = (df.groupby(["client_id"])['reading_remarque'].min()).to_list()

# Max Reading Remark
FINAL_DF['max_reading_remark'] = (df.groupby(["client_id"])['reading_remarque'].max()).to_list()

# Mean Reading Remark
FINAL_DF['mean_reading_remark'] = (df.groupby(["client_id"])['reading_remarque'].mean()).to_list()

In [ ]:
# Mean Index Difference
FINAL_DF['mean_difference_index'] = (df.groupby(["client_id"])['difference_index'].mean()).to_list()

# Sum Index Difference
FINAL_DF['sum_difference_index'] = (df.groupby(["client_id"])['difference_index'].sum()).to_list()

# Min Index Difference
FINAL_DF['min_difference_index'] = (df.groupby(["client_id"])['difference_index'].min()).to_list()

# Max Index Difference
FINAL_DF['max_difference_index'] = (df.groupby(["client_id"])['difference_index'].max()).to_list()

# Min Counter Status
FINAL_DF['min_counter_statue'] = (df.groupby(["client_id"])['counter_statue'].min()).to_list()

# Max Counter Status
FINAL_DF['max_counter_statue'] = (df.groupby(["client_id"])['counter_statue'].max()).to_list()

# Mean Counter Status
FINAL_DF['mean_counter_statue'] = (df.groupby(["client_id"])['counter_statue'].mean()).to_list()


In [ ]:
# Mean of Differences of each Counter Coefficient
new_df = df.groupby(["client_id"])["counter_coefficient"].apply(np.array).reset_index(name="counter_coefficient")
new_df["diff_counter_coefficient"] = new_df.apply(lambda x : np.diff(x["counter_coefficient"]) if (len(x["counter_coefficient"]) > 1) else x["counter_coefficient"], axis=1)
FINAL_DF["mean_diff_counter_cofficient"] = new_df.apply(lambda x : x["diff_counter_coefficient"].mean(), axis=1).to_list()

# Sum Counter Coefficient
FINAL_DF["sum_counter_coefficient"] = (df.groupby(["client_id"])["counter_coefficient"].sum()).to_list()

# Mean Counter Coefficient
FINAL_DF["mean_counter_coefficient"] = (df.groupby(["client_id"])["counter_coefficient"].mean()).to_list()

# Min Counter Coefficient
FINAL_DF["min_counter_coefficient"] = (df.groupby(["client_id"])["counter_coefficient"].min()).to_list()

# Max Counter Coefficient
FINAL_DF["max_counter_coefficient"] = (df.groupby(["client_id"])["counter_coefficient"].max()).to_list()

In [ ]:
consumption_levels = ["consommation_level_1", "consommation_level_2", "consommation_level_3", "consommation_level_4"]
client_consumption = df.groupby(["client_id"])[consumption_levels]

sum_total_consumption = client_consumption.sum().sum(axis = 1)
mean_total_consumption = client_consumption.mean().sum(axis = 1)
min_total_consumption = client_consumption.min().min(axis = 1)
max_total_consumption = client_consumption.max().max(axis = 1)

# Sum of Total Consumption
FINAL_DF["sum_total_consumption"] = sum_total_consumption.to_list()

# Mean of Total Consumption
FINAL_DF["mean_total_consumption"] = mean_total_consumption.to_list()

#Min Total Consumption
FINAL_DF["min_total_consumption"] = min_total_consumption.to_list()

#Max Total Consumption
FINAL_DF["max_total_consumption"] = max_total_consumption.to_list()


In [ ]:
df["total_consumption"] = df["consommation_level_1"] + df["consommation_level_2"] + df["consommation_level_3"] + df["consommation_level_4"]

df["tally_value"] = df["difference_index"] - df["total_consumption"]

# Check Invoice Tally is True
df["tally_check_true"] = df["tally_value"].apply(lambda x: 1 if (x == 0) else 0)

# Check Invoice Tally is False
df["tally_check_false"] = df["tally_value"].apply(lambda x: 1 if (x != 0) else 0)

# Sum of Tallied Invoices
FINAL_DF["tally_check_true"] = (df.groupby(["client_id"])["tally_check_true"].sum()).to_list()

# Sum of Non-tallied Invoices
FINAL_DF["tally_check_false"] = (df.groupby(["client_id"])["tally_check_false"].sum()).to_list()

# Sum Tally Value
FINAL_DF["sum_tally_value"] = (df.groupby(["client_id"])["tally_value"].sum()).to_list()

# Min Tally Value
FINAL_DF["min_tally_value"] = (df.groupby(["client_id"])["tally_value"].min()).to_list()

# Max Tally Value
FINAL_DF["max_tally_value"] = (df.groupby(["client_id"])["tally_value"].max()).to_list()

# Mean Tally Value
FINAL_DF["mean_tally_value"] = (df.groupby(["client_id"])["tally_value"].mean()).to_list()


In [ ]:
counters = df.groupby(["client_id"])["counter_type"]

check_same = lambda x: [True, x.mode()[0]] if (x.nunique() == 1) else [False, "BOTH"]
classification = lambda x: 0 if (x[0] and x[1] == "GAZ") else 1 if (x[0] and x[1] == "ELEC") else 2 

counter_type = counters.apply(check_same).apply(classification)

#Type of counter, 0 -> Gas, 1 -> Electricity, 2 -> Both
FINAL_DF["counter_type"] = counter_type.to_list()

In [ ]:
client_dates = df.groupby(["client_id"])["invoice_date"]

last_invoice = client_dates.max()

dates = pd.to_datetime(last_invoice)

last_year = dates.dt.year
last_month = dates.dt.month
last_day = dates.dt.day

# Year of Last Invoice
FINAL_DF["last_year"] = last_year.to_list()

# Month of Last Invoice
FINAL_DF["last_month"] = last_month.to_list()

# Day of Last Invoice
FINAL_DF["last_day"] = last_day.to_list()

# Is the Last Invoice on a Weekday, 0 -> No, 1 -> Yes
last_day_is_weekday = pd.DataFrame(((pd.DatetimeIndex(dates).dayofweek) <= 5).astype(int))
FINAL_DF["last_day_is_weekday"] = last_day_is_weekday


In [ ]:
# df["invoice_month"] = pd.to_datetime(df["invoice_date"], dayfirst = True).dt.month

# # Mode of Months of Invoices
# FINAL_DF["mode_month"] = (df.groupby(["client_id"])["invoice_month"].agg(pd.Series.mode)).to_list()

### FINAL DATAFRAME
Check if it is okay and the data types is correct before continuing.

In [ ]:
FINAL_DF

,client_id,district,client_catg,region,region_group,creation_date_day,creation_date_month,creation_date_year,no_months_as_client,services_consumed,...,tally_check_false,sum_tally_value,min_tally_value,max_tally_value,mean_tally_value,counter_type,last_year,last_month,last_day,last_day_is_weekday
0,train_Client_0,60,11,101,2,31,12,1994,290.568595,1,...,0,0,0,0,0.000000,1,2019,3,19,1
1,train_Client_1,69,11,107,2,29,5,2002,202.123247,1,...,0,0,0,0,0.000000,1,2019,4,2,1
2,train_Client_10,62,11,301,3,13,3,1986,397.642662,1,...,0,0,0,0,0.000000,1,2019,5,2,1
3,train_Client_100,69,11,105,2,7,11,1996,190.591183,1,...,0,0,0,0,0.000000,1,2012,9,25,1
4,train_Client_1000,62,11,303,3,14,10,2014,56.083287,1,...,0,0,0,0,0.000000,1,2019,6,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135482,train_Client_99995,62,11,304,3,26,7,2004,178.993408,2,...,1,-139,-139,0,-1.957746,2,2019,6,26,1
135483,train_Client_99996,63,11,311,3,25,10,2012,79.738804,2,...,0,0,0,0,0.000000,2,2019,6,18,1
135484,train_Client_99997,63,11,311,3,22,11,2011,95.476293,2,...,0,0,0,0,0.000000,2,2019,11,6,1
135485,train_Client_99998,60,11,101,2,22,12,1993,143.904392,1,...,0,0,0,0,0.000000,1,2005,12,19,1


In [ ]:
FINAL_DF.dtypes

client_id                        object
district                          int64
client_catg                       int64
region                            int64
region_group                      int64
creation_date_day                 int64
creation_date_month               int64
creation_date_year                int64
no_months_as_client             float64
services_consumed                 int64
target                            int64
months_of_service               float64
number_of_counter                 int64
number_of_instances               int64
number_of_person_counting         int64
min_reading_remark                int64
max_reading_remark                int64
mean_reading_remark             float64
mean_difference_index           float64
sum_difference_index              int64
min_difference_index              int64
max_difference_index              int64
min_counter_statue                int64
max_counter_statue                int64
mean_counter_statue             float64


### IT IS NOW TIME TO EXPORT TO CSV!!!!
## **Congratulations!**

In [ ]:
for i in FINAL_DF.columns:
  print(str(i)+":"+str(FINAL_DF[i].isnull().values.any()))

client_id:False
district:False
client_catg:False
region:False
region_group:False
creation_date_day:False
creation_date_month:False
creation_date_year:False
no_months_as_client:False
services_consumed:False
target:False
months_of_service:False
number_of_counter:False
number_of_instances:False
number_of_person_counting:False
min_reading_remark:False
max_reading_remark:False
mean_reading_remark:False
mean_difference_index:False
sum_difference_index:False
min_difference_index:False
max_difference_index:False
min_counter_statue:False
max_counter_statue:False
mean_counter_statue:False
mean_diff_counter_cofficient:False
sum_counter_coefficient:False
mean_counter_coefficient:False
min_counter_coefficient:False
max_counter_coefficient:False
sum_total_consumption:False
mean_total_consumption:False
min_total_consumption:False
max_total_consumption:False
tally_check_true:False
tally_check_false:False
sum_tally_value:False
min_tally_value:False
max_tally_value:False
mean_tally_value:False
counter_t

In [ ]:
FINAL_DF.to_csv('fixed_data_train.csv')